To run this example, you need to install:
    - python3
    - jupyter
    - the followign pyhton libraries (using pip): openpyxl, cfuzzyset
    - save the ontogies files in the jupyter notebook folder

## Create the mappings between the descriptors and TO or CO

In [189]:
import openpyxl
import json
import time
from cfuzzyset import cFuzzySet as FuzzySet
from IPython.core.display import HTML
import d3_lib
import re
import urllib.request

#### Import your excel file

In [180]:
## path to the file containing the descriptor information and list
## CHANGE THE PATH HERE
path = "/Users/marie-angeliquelaporte/Downloads/descriptorlist_WorldVeg.xlsx"

In [181]:
# open the file
wb = openpyxl.load_workbook(path)

In [182]:
# get to the desfriptors sheet - change the name of the sheet if necessary
worksheet = wb.get_sheet_by_name('descriptors')

In [183]:
# get the descriptors names
descriptors = []
descriptorsList = []
#test = []
#iterate over the rows in the specific column, here B = Title
for row in range(2,worksheet.max_row+1):  
    for column in "B":  #Here you can change the column you want to do the mapping on
        cell_name = "{}{}".format(column, row)
        value = worksheet[cell_name].value # the value of the specific cell
        descriptors.append(value)

#clean the descriptors list
for descriptor in descriptors:
    clean = re.sub('[!@#$.:,]', '', descriptor)
    clean = re.split('\(', clean)[0]
    clean = clean.strip()
    descriptorsList.append({"descriptor": descriptor, "clean": clean })


In [184]:
#print the descriptor list
print(descriptorsList)

[{'descriptor': 'Panicle: length (late observation)', 'clean': 'Panicle length'}, {'descriptor': 'tutu', 'clean': 'tutu'}, {'descriptor': 'toto', 'clean': 'toto'}, {'descriptor': 'Panicle length', 'clean': 'Panicle length'}, {'descriptor': 'Panicle: length', 'clean': 'Panicle length'}]


#### Load the ontology

In [185]:
# list of the possible ontologies : all CO and TO
## CHANGE HERE the name of ontologie you want to map the descriptors to: e.g. CO_320.json if rice, to.json if TO, ...
ontoLabels = []
ontoDict = {}
with open('data/CO_320.json') as data_file:
    onto = json.load(data_file)
    
    results = onto["results"]["bindings"]
    for entry in results:
        if "to" in entry:
            uri = entry["to"]["value"]
            label = entry["label"]["value"]
            ontoLabels.append(label)
            ontoDict[label] = uri
        else: 
            uri = entry["co"]["value"]
            label = entry["label"]["value"]
            ontoLabels.append(label)
            ontoDict[label] = uri

#### Create the mappings

In [186]:
##MAPPINGS CREATION
mappings = []
a=FuzzySet() # allocate the FuzzySet object

for e in ontoLabels:
    a.add(e)
    
start_time = time.time() # Start recording the time

for names in descriptorsList:
    if names["clean"]:
        if a.get(names["clean"])[0][0] > 0.6:  ##### CHANGE THE THRESHOLD HERE
            mappings.append({"descriptor": names["descriptor"], "onto label": a.get(names["clean"])[0][1] , "score": a.get(names["clean"])[0][0] , "uri": ontoDict[a.get(names["clean"])[0][1]]})
        else: 
            mappings.append({"descriptor": names["descriptor"], "onto label": "" , "score": "" , "uri": ""})
print("--- %s seconds ---" % (time.time() - start_time)) 
    
#print(mappings)

--- 0.00042891502380371094 seconds ---


## Results

In [187]:
#display results as a Table
# visualization using d3.js
HTML(d3_lib.set_styles(['table']) + 
'<script src="lib/d3/d3.min.js"></script>' + 
      d3_lib.draw_graph('table', {'data': mappings}) )

## Help - CO IDs

In [192]:
#CO cheat list
## list the CO IDs
url = "http://www.cropontology.org/get-ontologies"
r = urllib.request.urlopen(url)
data = json.loads(r.read().decode(r.info().get_param('charset') or 'utf-8'))

ontologies = data["300-499 Phenotype and Trait Ontology"]
ontologyList = []
for ontology in ontologies:
    if "CO_" in ontology["ontology_id"]:
        ontologyList.append({"id": ontology["ontology_id"] ,"name": ontology["ontology_name"]})

print(json.dumps(ontologyList, indent=2))
   

[
  {
    "name": "Banana",
    "id": "CO_325"
  },
  {
    "name": "Barley Trait Dictionary",
    "id": "CO_323"
  },
  {
    "name": "Brachiaria",
    "id": "CO_345"
  },
  {
    "name": "Cassava",
    "id": "CO_334"
  },
  {
    "name": "Castor bean",
    "id": "CO_347"
  },
  {
    "name": "Chickpea",
    "id": "CO_338"
  },
  {
    "name": "Common bean",
    "id": "CO_335"
  },
  {
    "name": "Cowpea",
    "id": "CO_340"
  },
  {
    "name": "Groundnut",
    "id": "CO_337"
  },
  {
    "name": "Lentil",
    "id": "CO_339"
  },
  {
    "name": "Maize",
    "id": "CO_322"
  },
  {
    "name": "Mungbean",
    "id": "CO_346"
  },
  {
    "name": "Oat",
    "id": "CO_350"
  },
  {
    "name": "Pearl millet",
    "id": "CO_327"
  },
  {
    "name": "Pigeonpea",
    "id": "CO_341"
  },
  {
    "name": "Potato",
    "id": "CO_330"
  },
  {
    "name": "Rice",
    "id": "CO_320"
  },
  {
    "name": "Sorghum",
    "id": "CO_324"
  },
  {
    "name": "Soybean",
    "id": "CO_336"
  },
  {
